Test 2 - Mercury Precession
--

This test aims to use the general relativity module (`gr.py`) in `wh5470` to verify the famous discrepancy of 43 seconds of arc per century in Mercury precession, which was first
explained by Einstein’s General Relativity theory in 1915.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from wh5470 import wh5470

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# constants
AU_to_meter = 1.496e+11
yr = 3600*24*365 # yr in s
G = 6.67430e-11  # Gravitational constant [N m^2 / kg^2]
c = 299792458.0  # Speed of light [m/s]
M_sun = 1.989e30 # Solar mass in [kg]
M_Jupiter = 1.898e27 # Jupiter mass in [kg]
M_Mercury = 3.285e23 # Mercury mass in [kg]

Similarly, we set up bodies and their initial positions and velocities. This will be a two-body problem, and simplified parameters will be taken since we only want to verify the discrepency precession rate caused by the gr effects from the Sun. Note we have to take into account the ~ 7 degree inclination of Mercury's orbit.

In [5]:
masses = np.array([M_sun, M_Mercury])


positions_initial = np.array([[0., 0., 0.], # Sun at origin
                      [0.4*AU_to_meter, 0., 0.]]) # Mercury at 5.2 AU from the Sun)  # test particle in an outer orbit

velocities_initial = np.array([[0., 0., 0.], # Sun initially stationary
                       [0., 58541, 7192]])

dt = 10*3600 # timestep
endtime = 100 * yr # total integration time

# add 'gr=True'
sim = wh5470(masses, positions=positions_initial, velocities=velocities_initial, time_step=dt, total_time=endtime, gr=True)


In [6]:
%%time
trajectory, velocity, energy, angular_momentum, orbital_elements, trajectory_jacobi, velocity_jacobi = sim.integrate()

CPU times: user 1min 6s, sys: 225 ms, total: 1min 6s
Wall time: 1min 6s


Calculate the precession rate using orbital elements ($\omega$).

In [7]:
omega_values = []
for oe in orbital_elements:
    omega_values.append(oe[1][4])

delta_omega = omega_values[-100] - omega_values[0]

In [8]:
precession_rate_rad_per_year = delta_omega / (endtime / (365 * 86400.0))  # Assuming total_time is in seconds
precession_rate_deg_per_century = np.rad2deg(precession_rate_rad_per_year) * 100

In [9]:
theoretical_precession_rate = 42.9799  # Theoretical precession rate of Mercury's perihelion (in arcseconds per century)

print(f"Precession rate from simulation: {precession_rate_deg_per_century:.4f} arcseconds per century")
print(f"Theoretical precession rate: {theoretical_precession_rate} arcseconds per century")

Precession rate from simulation: 69.8529 arcseconds per century
Theoretical precession rate: 42.9799 arcseconds per century


Since we are using a simplified model, and the gr modification function is not developed to full orders, this would be a more qualitative-verifying result then.